In [2]:
import requests, urllib2, gzip, urllib
import pandas as pd
import numpy as np

In [3]:
url = 'http://s3.amazonaws.com/thedataincubator/coursedata/mldata/train.txt.gz'

In [4]:
#Download file
urllib.urlretrieve(url,'Weather')

weather_list = []
with gzip.open('Weather', 'rb') as f:
    for line in f.readlines():
        info_dict = {}
        info = line.split()
        info_dict['month'] = info[1]
        info_dict['hour'] = info[3]
        info_dict['temp'] = int(info[4])
        info_dict['city'] = info[12]
        if info_dict['temp'] != -9999.0:
            weather_list.append(info_dict)
        
print weather_list[0:2]
print len(weather_list)


[{'city': 'bos', 'temp': -11, 'hour': '00', 'month': '01'}, {'city': 'bos', 'temp': -6, 'hour': '01', 'month': '01'}]
515581


In [5]:
weather_df = pd.DataFrame(weather_list)
weather_df.head(10)
weather_df.describe()

,temp
count,515581.000000
mean,121.691294
std,103.047547
min,-272.000000
25%,39.000000
50%,122.000000
75%,206.000000
max,406.000000


In [7]:
predict_df = weather_df.groupby(['city', 'hour', 'month']).agg({'temp': [np.mean]}).reset_index()

predict_df.columns = ['city', 'hour', 'month', 'av_temp']

#predict_df.head()

predict_df.describe()


,av_temp
count,1440.000000
mean,120.865884
std,91.578820
min,-58.013441
25%,38.106266
50%,119.553763
75%,202.966935
max,294.868280


In [19]:
predict_list = predict_df.values.tolist()
estimator_dict = {}
for item in predict_list:
    estimator_dict[(item[0], item[1], item[2])] = item[3]  
estimator_dict['Other'] = 121

    
print len(estimator_dict)


1441


In [20]:
class Estimator(object):
    def __init__(self, dict):
        self.dict = dict
    def predict(self, record):
        line = record.split()
        record_entry  = (line[12], line[3], line[1])
        if record_entry in self.dict:
            prediction = self.dict[record_entry]
        else:
            prediction = self.dict['Other']
        return prediction
    
estimator = Estimator(estimator_dict)


In [21]:
#Store the instance using dill.
import dill
with open('temp_est_file','w') as f:
    dill.dump(estimator,f)
    

In [22]:
test = "2000 01 01 02   xx   -78 10211   230    36     0     0 -9999 bos"

check = test.split()
check_entry = (check[12], check[3], check[1])
print check_entry

answer = estimator.predict(test)
print answer


('bos', '02', '01')
-12.7768817204


In [23]:
check2 = estimator_dict[(u'bos', u'00', u'01')]
print check2

-5.21505376344
